### 동시성

손님들 100명이 우체국에 일렬로 줄을 서있고, 한명씩 처리하는것 -> 싱글스레드 / 동기 처리
손님들 100명이 우체국의 100명의 직원에게 각각 처리하는 것 -> 멀티쓰레드 / 동기 처리

손님들 100명이 우체국 뒷마당에다가 소포를 던져두고, 이벤트 알림표를 받고 집에가서 자기 할일을 하는것. -> 비동기


### 병렬성
1. 일반 쓰레드를 사용하는 방식
2. 일반 프로세스를 사용하는 방식
3. 여러 컴퓨터에 분산해서 사용하는 방식 
4. GPGPU 를 통한 방식 (그래픽카드의 수만개 이상의 쓰레딩모듈) 
5. CPU 를 통한 방식 (SIMD 명령방식 : 단일명령으로 다중데이터처리) 

## 파이썬에서의 동시성
- 멀티 쓰레드
    - 파이썬에서는 GIL이라는 것이 존재해 cpu를 하나만 사용하게 됨. 병렬계산할때는 오히려 더 성능이 안좋아짐.(쓰레드를 무작정 늘리는건 좋지 않다.)
    - 이때문에, 쓰레드를 활용할때는 I/O, 비동기에서만!!
    - 병렬 계산 할때는 프로세스, GPU, 분산을 활용
- 멀티 프로세스
    - Queue를 이용할때 `import Queue`가 아닌 `from multiprocessing import Queue`모듈을 이용해야함
- EXECUTORS 와 퓨처(FUTURE)
- 비동기와 ASYNCIO
    - 파이썬은 GIL에 의해 한번에 하나의 쓰레드 위주로 작동하지만, 로우레벨로 내려가면 GIL 을 무시하고 자체적인 I/O 실행환경을 활용하게 됨.
    - 하나의 쓰레드가 I/O 작업은 OS에게 맞겨 두고 자신의 일을 하다가, OS 가 어떤 이벤트를 알려오면 ( 나 일 끝났어요~~ 등등) 그때 제어권을 살짝 바꾸어서 그 이벤트에 해당하는 일을 하게 하는 것입니다.
    - 비동기를 알려면 제너레이터와 코루틴을 알아야함

## 제너레이터

In [79]:
def simple_gen():
    yield "Hello"
    yield "World"


gen = simple_gen()
print(next(gen))
print(next(gen))

Hello
World


- simple_gen 함수가 수행되다가 yield를 만나게 되면 해당 값을 반환. 그리고 끝나는게 아니라 그 상태에서 머물게 됨
- 보통의 함수는 호출되면 return을 만나거나 블록의 끝까지 수행되고 나면 끝
- 제너레이터는 next()로 함수를 yield까지 가게해서 계속 함수를 수행시킬 수 있음. 그리고 더이상 yield까지 갈 수 없을 경우 `StopIteration`예외를 내면서 끝남
- 즉, 제너레이터를 활용해서 함수에서 데이터를 가져올 수 있고 실행을 일시 중지 할 수 있음.

## 코루틴
- 보통 함수는 입력으로 주어진 인수에 대해서 한 번만 실행됨.
- 제너레이터를 확장하는 방법으로 구현
- 한번 활성화 되면 소진될 때까지 1kb 미만의 메모리만 소비
- 제네레이터 자신이 계속 데이터를 만드는 것이 아니라, 외부에서 제공되는 데이터를 소비하는 역할을 하는 것.
- 코루틴은 yield로 값을 받아 올 수 있음
- send() 메서드를 사용하여 값을 함수로 다시 전달할 수 있음 -> "제네레이터 기반 코루틴" 이라고 함

In [60]:
def coro():
    hello = yield "Hello"
#     yield hello
    print(hello)


c = coro()
print(next(c)) # 코루틴을 준비함
print(c.send("World"))  # send를 이용해 코루틴 함수로 값을 넣어줄 수 있음. 
print(c.send("bye"))

Hello
World


StopIteration: 

비동기에서 코루틴은 send를 이용하여 비동기 작업에 대한 결과를 받아서 전달하는 매개함수 역할을 함

코루틴을 종료하지 않고 계속 유지시키기 위해 기본적으로 무한 루프를 이용

In [64]:
# 다른 예제
def print_matches(matchtext):
    print("Looking for", matchtext)  #3
    while True:   #4
        line = (yield) #5
        if matchtext in line: #7
            print(line) #8
            
match = print_matches('good')   #1
next(match) #2. 코루틴을 준비함. 코루틴 안의 yield까지 코드 실행(최초 실행) 
match.send('python is good')  #6
match.send('other is also good')
match.close()
match.send('bad')

Looking for good
python is good
other is also good


StopIteration: 

1. 메인 루틴의 #1, #2이 실행
2. print_matches 코루틴 #3, #4, #5가 실행되며, #5의 yield에서 대기하며 다시 메인루틴으로 돌아옴
3. 메인 루틴의 #6이 실행
4. print_matches 코루틴 #5, #7, #8, #5가 실행되며, #5의 yield에서 대기하며 다시 메인루틴으로 돌아옴
5. 반복

즉, 코루틴은 send()로 값이 도착할 때까지 멈춰 있음.  
send()가 호출되면, 코루틴 안에서 (yield) 표현식에 의해 값이 반환되고 바로 다음 문장에 의해 처리됨. 
다음 (yield) 표현식을 만날 때까지 계속되고, 그 순간 멈춘 함수가 다시 멈춤.

In [96]:
# 다른 예제
def sum_coroutine():
    total = 0
    while True:
        x = (yield total)    # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        total += x
 
co = sum_coroutine()
print(next(co))      # 0: 코루틴 안의 yield까지 코드를 실행하고 코루틴에서 나온 값 출력
 
print(co.send(1))    # 1: 코루틴에 숫자 1을 보내고 코루틴에서 나온 값 출력
print(co.send(2))    # 3: 코루틴에 숫자 2를 보내고 코루틴에서 나온 값 출력
print(co.send(3))    # 6: 코루틴에 숫자 3을 보내고 코루틴에서 나온 값 출력

0
1
3
6


In [98]:
# 다른 예제
def minimize():
    current = yield
    while True:
        value = yield current
        current = min(value, current)

# 제너레이터를 소비하는 코드는 한 번에 한 단계씩 실행하여 각 입력을 받은 이후의 최솟값을 출력
it = minimize()
next(it)  # 제너레이터를 준비함
print(it.send(10))
print(it.send(7))
print(it.send(14))
print(it.send(-1))

10
7
7
-1


send를 새로 호출할 때마다 전진하면서 계속 실행하는 것처럼 보임.  
스레드와 마찬가지로 코루틴은 주변 환경에서 받은 입력을 소비하여 결과를 만들어낼 수 있는 독립적인 함수.  
다만, 코루틴은 코루틴이 제너레이터 함수의 각 yield 표현식에서 멈췄다가 외부에서 send를 호출할 때마다 다시 시작한다는 점이 다름.  
위 동작 덕분에 제너레이터를 소비하는 코드에서 코루틴의 각 yield 표현식 이후에 원하는 처리를 할 수 있음.

#### GeneratorExit 예외 처리하기
- close 메서드를 호출해서 코루틴이 종료될 때, GeneratorExit 예외가 발생.  
- 아래와 같이 예외 처리를 해서 코루틴의 종료 시점을 알 수 있음

In [99]:
def number_coroutine():
    try:
        while True:
            x = (yield)
            print(x, end=' ')
    except GeneratorExit:    # 코루틴이 종료 될 때 GeneratorExit 예외 발생
        print()
        print('코루틴 종료')
 
co = number_coroutine()
next(co)
 
for i in range(20):
    co.send(i)
 
co.close()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료


In [102]:
def sum_coroutine():
    try:
        total = 0
        while True:
            x = (yield)
            total += x
    except RuntimeError as e:
        print(e)
        yield total    # 코루틴 바깥으로 값 전달
 
co = sum_coroutine()
next(co)
 
for i in range(20):
    co.send(i)
 
print(co.throw(RuntimeError, '예외로 코루틴 끝내기')) # 190
                                                      # 코루틴의 except에서 yield로 전달받은 값

예외로 코루틴 끝내기
190


#### 코루틴 초기화를 자동화 하기

In [103]:
def coroutine(func):    # 코루틴 초기화 데코레이터
    def init(*args, **kwargs):
        co = func(*args, **kwargs)    # 코루틴 객체 생성
        next(co)                      # next 호출
        return co                     # 코루틴 객체 반환
    return init
 
@coroutine    # 코루틴 초기화 데코레이터 지정
def sum_coroutine():
        total = 0
        while True:
            x = (yield total)
            total += x
 
co = sum_coroutine()    # 코루틴 객체를 생성한 뒤 바로 사용
 
print(co.send(1))
print(co.send(2))
print(co.send(3))

1
3
6


## Async I/O
-  비동기 프로그래밍을 위한 API를 제공하는 asyncio 모듈
- asyncio 모듈을 이용하면 coroutines를 사용하여 비동기 작업을 쉽게 수행 가능

'''전체 시스템이 블러킹 안되게 하는 방법
    1. 멀티쓰레드를 통해 하나만 블럭되게 한다
    2. __비동기 방식을 사용__

In [80]:
import asyncio
import datetime
import random


@asyncio.coroutine
def display_date(num, loop):
    end_time = loop.time() + 50.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        yield from asyncio.sleep(random.randint(0, 5))  #


loop = asyncio.get_event_loop() # 이벤트 루프를 초기화한다.

asyncio.ensure_future(display_date(1, loop))   # 코루틴(async def로 정의된)을 asyncio가 스케쥴링 할 수 있는 Task 오브젝트(Future의 서브클래스)로 wrapping 해준다
asyncio.ensure_future(display_date(2, loop))

# loop.run_forever()
loop.run_until_complete()  # asyncio.wait()가 'done'이라는 Future를 반환할때까지 계속 실행된다

TypeError: run_until_complete() missing 1 required positional argument: 'future'

Loop: 1 Time: 2019-06-22 09:10:12.699633
Loop: 2 Time: 2019-06-22 09:10:12.699700
Loop: 1 Time: 2019-06-22 09:10:13.703270
Loop: 2 Time: 2019-06-22 09:10:13.703498
Loop: 1 Time: 2019-06-22 09:10:14.706639
Loop: 2 Time: 2019-06-22 09:10:16.706394
Loop: 1 Time: 2019-06-22 09:10:16.707186
Loop: 1 Time: 2019-06-22 09:10:18.710772
Loop: 1 Time: 2019-06-22 09:10:20.712428
Loop: 1 Time: 2019-06-22 09:10:20.712595
Loop: 2 Time: 2019-06-22 09:10:21.706843
Loop: 2 Time: 2019-06-22 09:10:21.707010
Loop: 2 Time: 2019-06-22 09:10:22.710830
Loop: 1 Time: 2019-06-22 09:10:22.712941
Loop: 1 Time: 2019-06-22 09:10:22.713194
Loop: 2 Time: 2019-06-22 09:10:23.711806
Loop: 2 Time: 2019-06-22 09:10:23.712049
Loop: 2 Time: 2019-06-22 09:10:23.712141
Loop: 1 Time: 2019-06-22 09:10:24.715278
Loop: 2 Time: 2019-06-22 09:10:27.713834
Loop: 1 Time: 2019-06-22 09:10:29.717935
Loop: 1 Time: 2019-06-22 09:10:30.722534
Loop: 2 Time: 2019-06-22 09:10:31.718079
Loop: 1 Time: 2019-06-22 09:10:31.724006
Loop: 2 Time: 20

- async def는 코루틴을 정의하기 위한 키워드.
- asyncio.sleep()은 코루틴이므로 여기서의 await는 다른 코루틴이 결과값을 만들때까지 기다리는 용도.
- 또한 await future인 경우에도 코루틴이 ensure_future가 작업을 완료할 때까지 기다리다가 값을 받아옴.
- await 뒤에 오는 함수도 코루틴으로 작성되어 있어야함.

In [110]:
import asyncio
import datetime
import random

async def display_date(num, loop, ):  # <----- 요기
    end_time = loop.time() + 50.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(random.randint(0, 5)) # <----- 요기 


loop = asyncio.get_event_loop()

asyncio.ensure_future(display_date(1, loop))
asyncio.ensure_future(display_date(2, loop))

# loop.run_forever()
loop.run_until_complete()

TypeError: run_until_complete() missing 1 required positional argument: 'future'

Loop: 1 Time: 2019-06-22 11:56:55.151583
Loop: 2 Time: 2019-06-22 11:56:55.151751
Loop: 2 Time: 2019-06-22 11:56:55.151922
Loop: 1 Time: 2019-06-22 11:56:57.154205
Loop: 1 Time: 2019-06-22 11:56:57.154323
Loop: 2 Time: 2019-06-22 11:56:58.155124
Loop: 2 Time: 2019-06-22 11:56:58.155220
Loop: 2 Time: 2019-06-22 11:56:59.159933
Loop: 1 Time: 2019-06-22 11:57:00.155694
Loop: 1 Time: 2019-06-22 11:57:01.160998
Loop: 2 Time: 2019-06-22 11:57:03.163940
Loop: 2 Time: 2019-06-22 11:57:03.164184
Loop: 2 Time: 2019-06-22 11:57:04.165977
Loop: 1 Time: 2019-06-22 11:57:05.162175
Loop: 2 Time: 2019-06-22 11:57:06.167856
Loop: 1 Time: 2019-06-22 11:57:07.163976
Loop: 2 Time: 2019-06-22 11:57:07.169193
Loop: 1 Time: 2019-06-22 11:57:09.167927
Loop: 2 Time: 2019-06-22 11:57:10.172207
Loop: 2 Time: 2019-06-22 11:57:11.175967
Loop: 2 Time: 2019-06-22 11:57:11.176210
Loop: 1 Time: 2019-06-22 11:57:13.172781
Loop: 1 Time: 2019-06-22 11:57:13.172919
Loop: 2 Time: 2019-06-22 11:57:14.178180
Loop: 2 Time: 20

### 동기적 방식과 비동기적 방식의 차이

In [112]:
# 동기적 처리
import time

def sub_routine_1():
    print('서브루틴 1 시작')
    print('서브루틴 1 중단... 5초간 대기')
    time.sleep(5)
    print('서브루틴 1 종료')

def sub_routine_2():
    print('서브루틴 2 시작')
    print('서브루틴 2 중단... 4초간 대기')
    time.sleep(4)
    print('서브루틴 2 종료')


sub_routines = [sub_routine_1, sub_routine_2]

start = time.time()
for i in range(2):
    sub_routines[i]()
end = time.time()
print(f'time taken: {end-start}')

서브루틴 1 시작
서브루틴 1 중단... 5초간 대기
서브루틴 1 종료
서브루틴 2 시작
서브루틴 2 중단... 4초간 대기
서브루틴 2 종료
time taken: 9.006868124008179


In [116]:
# 비동기적 처리
import asyncio
import time


async def coroutine_1():  # 코루틴 정의 (async를 앞에 붙여준다.)
    print('코루틴 1 시작')
    print('코루틴 1 중단... 5초간 대기')
    # await으로 중단점 설정 (블락킹되는 부분에서 사용)
    await asyncio.sleep(5)
    print('코루틴 1 재개')


async def coroutine_2():
    print('코루틴 2 시작')

    print('코루틴 2중단... 4초간 대기')
    await asyncio.sleep(4)
    print('코루틴 2 재개')


# 이벤트 루프 정의
loop = asyncio.get_event_loop()

start = time.time()

# 두 개의 코루틴을 이벤트 루프에서 돌린다.
# 코루틴이 여러개일 경우, asyncio.gather을 먼저 이용 (순서대로 스케쥴링 된다.)
loop.run_until_complete(asyncio.gather(coroutine_1(), coroutine_2()))
end = time.time()

# print(f'time taken: {end-start}')
print('time taken:', end-start)

RuntimeError: This event loop is already running

코루틴 1 시작
코루틴 1 중단... 5초간 대기
코루틴 2 시작
코루틴 2중단... 4초간 대기
코루틴 2 재개
코루틴 1 재개


---

## 비교하기
### 1. for loop를 이용하여 url 100개에 GET 요청 보내기

In [77]:
import requests
import timeit


url_basis = 'http://httpbin.org/get?'

start = timeit.default_timer()

for i in range(100):
    url = url_basis + str(i)
    print('Start', url)
    r = requests.get(url)
    print('Done', url)
    
duration = timeit.default_timer()-start
print(duration)

Start http://httpbin.org/get?0
Done http://httpbin.org/get?0
Start http://httpbin.org/get?1
Done http://httpbin.org/get?1
Start http://httpbin.org/get?2
Done http://httpbin.org/get?2
Start http://httpbin.org/get?3
Done http://httpbin.org/get?3
Start http://httpbin.org/get?4
Done http://httpbin.org/get?4
Start http://httpbin.org/get?5
Done http://httpbin.org/get?5
Start http://httpbin.org/get?6
Done http://httpbin.org/get?6
Start http://httpbin.org/get?7
Done http://httpbin.org/get?7
Start http://httpbin.org/get?8
Done http://httpbin.org/get?8
Start http://httpbin.org/get?9
Done http://httpbin.org/get?9
Start http://httpbin.org/get?10
Done http://httpbin.org/get?10
Start http://httpbin.org/get?11
Done http://httpbin.org/get?11
Start http://httpbin.org/get?12
Done http://httpbin.org/get?12
Start http://httpbin.org/get?13
Done http://httpbin.org/get?13
Start http://httpbin.org/get?14
Done http://httpbin.org/get?14
Start http://httpbin.org/get?15
Done http://httpbin.org/get?15
Start http:/

### 멀티 쓰레딩 이용

In [78]:
import threading
import queue
import requests
import timeit

thread_num = 100

def doWork():
    while True:
        print(threading.current_thread().name)
        url = q.get()
        print(threading.current_thread().name, url)
        r = requests.get(url)
        print(threading.current_thread().name, r.status_code)
        q.task_done()

start = timeit.default_timer()

q = queue.Queue(thread_num)

for i in range(thread_num):
    t = threading.Thread(target=doWork)
    t.daemon = True
    t.start()

for i in range(thread_num):
    q.put('http://httpbin.org/get?key=' + str(i))

q.join()
duration = timeit.default_timer() - start
print(duration)

# 먼저 쓰레드를 thread_num 만큼 doWork를 호출하도록 실행함.
# doWork가 호출되지만 아직 queue가 비어있는 상태이므로 쓰레드들은 계속 루프를 돈다. (queue가 비어있으면 lock을 걸어놓고 있다)
# queue에 thread_num 만큼의 URL을 넣는다.
# q.join()으로 쓰레드들이 계속 doWork의 무한루프를 돌게 한다.
# doWork에서 request를 수행하고 결과를 출력한 후 task_done()을 할 때마다 queue는 task가 처리된걸로 본다.
# 이렇게 thread_num 만큼의 job을 전부 수행하면 q.join()의 lock이 풀리고 프로그램이 종료된다.

Thread-104
Thread-105
Thread-106
Thread-107
Thread-108
Thread-109
Thread-110
Thread-111
Thread-112
Thread-113
Thread-114Thread-115

Thread-116
Thread-117Thread-118

Thread-119
Thread-120Thread-121

Thread-122
Thread-123Thread-124

Thread-125
Thread-126Thread-127

Thread-128
Thread-129Thread-130

Thread-131
Thread-132Thread-133

Thread-134
Thread-135
Thread-136Thread-137

Thread-138
Thread-139
Thread-140Thread-141

Thread-142
Thread-143Thread-144

Thread-145
Thread-146Thread-147

Thread-148
Thread-149Thread-150

Thread-151
Thread-152Thread-153

Thread-154
Thread-155
Thread-156
Thread-157Thread-158

Thread-159
Thread-160Thread-161

Thread-162Thread-163

Thread-164
Thread-165
Thread-166
Thread-167Thread-168

Thread-169
Thread-170Thread-171

Thread-172
Thread-173Thread-174

Thread-175
Thread-176Thread-177

Thread-178
Thread-179
Thread-180
Thread-181Thread-182

Thread-183
Thread-184Thread-185

Thread-186
Thread-187
Thread-188
Thread-189
Thread-190Thread-191

Thread-192
Thread-193Thread-194


### asyncio 이용

In [104]:
import asyncio
from aiohttp import ClientSession

async def hello(url):
    async with ClientSession() as session:
        async with session.get(url) as response:
            r = await response.read()
            print(r)

            loop = asyncio.get_event_loop()
tasks = []
url = 'http://httpbin.org/get?{0}'

start = timeit.default_timer()

for i in range(100):
    task = asyncio.ensure_future(hello(url.format(i)))  # 코루틴(async def로 정의된)을 asyncio가 스케쥴링 할 수 있는 Task 오브젝트(Future의 서브클래스)로 wrapping 해준다.
    tasks.append(task)

# run_until_complete는 asyncio.wait()가 'done'이라는 Future를 반환할때까지 계속 실행된다.
# loop.run_until_complete(asyncio.wait(tasks))  # asyncio.wait() 는 이 Task들의 list인 tasks가 전부 끝날때까지 기다린다.

duration = timeit.default_timer() - start
print('-'*40)
print(duration)


----------------------------------------
0.0010806569989654236
b'{\n  "args": {\n    "28": ""\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.5.4"\n  }, \n  "origin": "175.114.49.156, 175.114.49.156", \n  "url": "https://httpbin.org/get?28"\n}\n'
b'{\n  "args": {\n    "48": ""\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.5.4"\n  }, \n  "origin": "175.114.49.156, 175.114.49.156", \n  "url": "https://httpbin.org/get?48"\n}\n'
b'{\n  "args": {\n    "13": ""\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.6 aiohttp/3.5.4"\n  }, \n  "origin": "175.114.49.156, 175.114.49.156", \n  "url": "https://httpbin.org/get?13"\n}\n'
b'{\n  "args": {\n    "33": ""\n  }, \n  "headers": {\n

# 끝